# MNIST low level 2 layer CNN minimal working example

Low level example to show a basic implementation of a Convolutional Neural Network using TensorFlow. Most of the code is recycled from https://www.tensorflow.org/versions/r1.2/get_started/mnist/pros

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST", one_hot=True)

## Hyperparameters

In [ ]:
#not all... but some (Moste are still "hardcoded")
learn_rate = 1e-4
iterations = 10000
batch_size = 50

## Helper functions for Variable creation

In [ ]:

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)


def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

## Model definition

In [ ]:
'''
Placeholders: 
    -input_image is the "flat" one_hot representation of the input data.
    -input_labels is the correct classification of the corresponding input_image
'''
input_image = tf.placeholder(tf.float32, [None, 784])
input_labels = tf.placeholder(tf.float32, [None, 10])

input_image_reshaped = tf.reshape(input_image, [-1, 28, 28, 1])


#----------- Convolution Layer 1 ------------

filter_layer_1 = weight_variable([5, 5, 1, 32])
bias_layer_1 = bias_variable([32])
#Perform the convolution. conv1 contains the 32 feature maps (each of them created by one of the 32 filters!)
conv1 = tf.nn.conv2d(input_image_reshaped, 
                     filter_layer_1, 
                     strides=[1, 1, 1, 1], 
                     padding='SAME')
#Relu as activation function!
conv_layer_1 = tf.nn.relu(conv1 + bias_layer_1)

pooling_layer_1 = tf.nn.max_pool(conv_layer_1,
                                 ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1],
                                 padding='SAME')


#------------ Convolution Layer 2 -----------


filter_layer_2 = weight_variable([5, 5, 32, 64])
bias_layer_2 = bias_variable([64])
conv2 = tf.nn.conv2d(pooling_layer_1,
                     filter_layer_2,
                     strides=[1, 1, 1, 1], 
                     padding='SAME')

conv_layer_2 = tf.nn.relu(conv2 + bias_layer_2)

pooling_layer_2 = tf.nn.max_pool(conv_layer_2,
                                 ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1],
                                 padding='SAME')


#----------- Fully Connected Layer 1 -------------

# Pooling the input twice brings the size down to 7*7 !
weights_fc_1 = weight_variable([7 * 7 * 64, 1024])
bias_fc_1 = bias_variable([1024])

pooling_layer_2_flat = tf.reshape(pooling_layer_2, [-1, 7*7*64])
fc_1 = tf.nn.relu(tf.matmul(pooling_layer_2_flat, weights_fc_1) + bias_fc_1)

#------------ Dropout Layer --------------------
keep_prob = tf.placeholder(tf.float32)
dropout_layer = tf.nn.dropout(fc_1, keep_prob)

#------------ Fully Connected Layer 2 ----------

# Finally! The 2nd fully connected layer. The 10 output neurons, "output" 
# the probability of the image belonging to one of the 10 classes.
weights_fc_2 = weight_variable([1024, 10])
bias_fc_2 = bias_variable([10])
outplut_class_probabilities = tf.matmul(dropout_layer, weights_fc_2) + bias_fc_2


In [ ]:
# The error is beeing minimized by the optimizer during the training steps!
error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=input_labels, 
                                                               logits=outplut_class_probabilities))

optimizer = tf.train.AdamOptimizer(learn_rate)
train_step = optimizer.minimize(error)

correct_prediction = tf.equal(tf.argmax(outplut_class_probabilities, 1), tf.argmax(input_labels, 1))
correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)


## Start the TensorFlow session

In [ ]:
session = tf.Session()
session.run(tf.global_variables_initializer())

## Start training

In [ ]:
for i in range(iterations):
    batch = mnist.train.next_batch(batch_size)
    if i % 100 == 0:
        train_accuracy = session.run(accuracy,feed_dict={input_image: batch[0], input_labels: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    session.run(train_step,feed_dict={input_image: batch[0], input_labels: batch[1], keep_prob: 0.5})
    
    
    

## Print the classification accuracy on the test data

In [ ]:
print('test accuracy %g' % session.run(accuracy,feed_dict={input_image: mnist.test.images, input_labels: mnist.test.labels, keep_prob: 1.0}))

# Visualisation

In [ ]:
import matplotlib.pyplot as plt

#display all convolution filters of first convolution layer
#Those where learned by the nn during the training phase !
transposed_filters_layer1 = tf.transpose(filter_layer_1, [3, 0, 1, 2])

f, axarr = plt.subplots(4,8,figsize=(15, 15))
im_id = 0
f = 0
for i in range(4):
    for j in range(8):
        try:
            axarr[i,j].imshow(session.run(tf.reshape(transposed_filters_layer1[f], (5,5))))
            axarr[i,j].axis('off')
            im_id = im_id+1
            f +=1
        except:
            continue
plt.show()